# IMPORTANTE

Questo file viene utilizzato per provare le chiamate API sfruttando la camera fornita dal Cisco DTLab. 

Essendo più a diretto contatto con il DTLab, alcune funzionalità, come la creazione di un broker MQTT all'interno della rete, possono essere sperimentate più facilmente

Non rappresenta quindi il prodotto finale


### Cancella snapshot

In [1]:
### Cancella camera_snapshot
import os
import shutil

dir_name = "camera_snapshot"
abs_dir = os.path.abspath(dir_name)
cwd = os.path.abspath(os.getcwd())

if not os.path.exists(abs_dir):
    print(f"Directory not found: {abs_dir}")
else:
    # safety: only remove if the directory is inside the current working directory
    if os.path.commonpath([cwd, abs_dir]) != cwd:
        print(f"Refusing to delete directory outside the current working directory: {abs_dir}")
    else:
        try:
            shutil.rmtree(abs_dir)
            print(f"Deleted directory and all contents: {abs_dir}")
        except Exception as e:
            print(f"Error while deleting {abs_dir}: {e}")


Deleted directory and all contents: c:\Users\Simonee\Desktop\ACSM - Advanced Construction Site Monitoring\camera_snapshot


### Imports e credenziali ☑️

In [ ]:
import requests
import meraki
import credentials as cr
import time
import os

from pprint import pprint

API_KEY = cr.MERAKI_KEY_ABITAREIN_ADMIN
BASE_URL = "https://api.meraki.com/api/v1/organizations/"

dashboard = meraki.DashboardAPI(api_key=API_KEY, output_log=True, print_console=True, log_path='logs/') 
organizations = dashboard.organizations.getOrganizations()

2025-09-21 13:07:52       meraki:     INFO > Meraki dashboard API session initialized with these parameters: {'version': '2.0.3', 'api_key': '************************************4b76', 'base_url': 'https://api.meraki.com/api/v1', 'single_request_timeout': 60, 'certificate_path': '', 'requests_proxy': '', 'wait_on_rate_limit': True, 'nginx_429_retry_wait_time': 60, 'action_batch_retry_wait_time': 60, 'network_delete_retry_wait_time': 240, 'retry_4xx_error': False, 'retry_4xx_error_wait_time': 60, 'maximum_retries': 2, 'simulate': False, 'be_geo_id': None, 'caller': None, 'use_iterator_for_get_pages': False}
2025-09-21 13:07:52       meraki:     INFO > GET https://api.meraki.com/api/v1/organizations
2025-09-21 13:07:52       meraki:     INFO > organizations, getOrganizations; page 1 - 200 OK


### Organizzazione ☑️

In [2]:
ORG_ID = organizations[0]['id']
print (ORG_ID)

1194841


### Network ☑️

In [3]:
response_netID = dashboard.organizations.getOrganizationNetworks(ORG_ID)  
print(response_netID)
NETWORK_ID = response_netID[0]['id']

print(NETWORK_ID)

2025-09-21 13:07:54       meraki:     INFO > GET https://api.meraki.com/api/v1/organizations/1194841/networks
2025-09-21 13:07:54       meraki:     INFO > organizations, getOrganizationNetworks; page 1 - 200 OK


[{'id': 'L_575897802350018094', 'organizationId': '1194841', 'name': 'THU', 'productTypes': ['appliance', 'camera', 'cellularGateway', 'sensor', 'switch', 'wireless'], 'timeZone': 'Europe/Rome', 'tags': ['THU'], 'enrollmentString': None, 'url': 'https://n23.meraki.com/THU-appliance/n/nD7Xvdx/manage/clients', 'notes': 'Viale Certosa, 75 Milano', 'isBoundToConfigTemplate': False, 'isVirtual': False}, {'id': 'L_575897802350019550', 'organizationId': '1194841', 'name': 'HOM_BIS', 'productTypes': ['appliance', 'camera', 'cellularGateway', 'sensor', 'switch', 'wireless'], 'timeZone': 'Europe/Rome', 'tags': ['HOM_BIS'], 'enrollmentString': None, 'url': 'https://n23.meraki.com/HOM_BIS-applianc/n/BMRb4dx/manage/clients', 'notes': None, 'isBoundToConfigTemplate': False, 'isVirtual': False}, {'id': 'L_575897802350020686', 'organizationId': '1194841', 'name': 'B12', 'productTypes': ['appliance', 'camera', 'cellularGateway', 'sensor', 'switch', 'wireless'], 'timeZone': 'Europe/Rome', 'tags': ['B12'

### Ottenimento seriali di tutti i dispositivi ☑️

In [4]:
response_serial = dashboard.organizations.getOrganizationDevices(ORG_ID)
print(response_serial)

2025-09-21 13:07:55       meraki:     INFO > GET https://api.meraki.com/api/v1/organizations/1194841/devices
2025-09-21 13:07:56       meraki:     INFO > organizations, getOrganizationDevices; page 1 - 200 OK


[{'name': 'HOM_BIS-MV93', 'serial': 'Q2LU-EDMJ-3X4W', 'mac': '08:f1:b3:e3:ae:65', 'networkId': 'L_575897802350019550', 'productType': 'camera', 'model': 'MV93', 'address': 'Via Bistolfi, 7 Milano', 'lat': 45.47216225053124, 'lng': 9.24452304840088, 'notes': '', 'tags': [], 'lanIp': '10.20.0.4', 'configurationUpdatedAt': '2025-08-03T02:00:19Z', 'firmware': 'camera-7-0', 'url': 'https://n23.meraki.com/HOM_BIS-camera/n/K-kinax/manage/nodes/new_list/9834198183525', 'details': [{'name': 'Running software version', 'value': 'MV 7.0'}]}, {'name': 'THU-MV93', 'serial': 'Q2LU-UZAF-F8ZH', 'mac': '08:f1:b3:e3:b0:15', 'networkId': 'L_575897802350018094', 'productType': 'camera', 'model': 'MV93', 'address': 'via bombay 1/5 roma', 'lat': 41.82262, 'lng': 12.44557, 'notes': '', 'tags': [], 'lanIp': '10.20.1.2', 'configurationUpdatedAt': '2025-09-07T14:07:11Z', 'firmware': 'camera-7-0', 'url': 'https://n23.meraki.com/THU-camera/n/pU8Ozcx/manage/nodes/new_list/9834198183957', 'details': [{'name': 'Runn

### Categorizzazione seriali dei dispositivi ☑️

In [5]:


SERIAL = response_serial[0]['serial']
DEVICE_MODEL = response_serial[0]['model']
DEVICE_PRODUCT = response_serial[0]['productType']

DEVICE = {}
i = 0
# To specify a range, use slicing. For example, to get the first 3 serials:
for device in response_serial[0:3]:
    DEVICE[i] = {
        "NUMBER": i,
        "SERIAL": device['serial'],
        "DEVICE_MODEL": device['model'],
        "DEVICE_PRODUCT": device['productType'],
        "MAC_ADDRESS" : device['mac'],
        "HAS_A_UPLINK" : None,
        "DEVICE_UPLINK" : None
    }
    i = i+1
pprint(DEVICE)


{0: {'DEVICE_MODEL': 'MV93',
     'DEVICE_PRODUCT': 'camera',
     'DEVICE_UPLINK': None,
     'HAS_A_UPLINK': None,
     'MAC_ADDRESS': '08:f1:b3:e3:ae:65',
     'NUMBER': 0,
     'SERIAL': 'Q2LU-EDMJ-3X4W'},
 1: {'DEVICE_MODEL': 'MV93',
     'DEVICE_PRODUCT': 'camera',
     'DEVICE_UPLINK': None,
     'HAS_A_UPLINK': None,
     'MAC_ADDRESS': '08:f1:b3:e3:b0:15',
     'NUMBER': 1,
     'SERIAL': 'Q2LU-UZAF-F8ZH'},
 2: {'DEVICE_MODEL': 'MV93',
     'DEVICE_PRODUCT': 'camera',
     'DEVICE_UPLINK': None,
     'HAS_A_UPLINK': None,
     'MAC_ADDRESS': '08:f1:b3:e3:ae:fe',
     'NUMBER': 2,
     'SERIAL': 'Q2LU-XAD9-GLD5'}}


### Dispositivi raggiungibili nella rete ☑️

In [6]:
response_uplink = dashboard.organizations.getOrganizationDevicesUplinksAddressesByDevice(ORG_ID, serials=[SERIAL])

pprint(response_uplink)

2025-09-21 13:07:58       meraki:     INFO > GET https://api.meraki.com/api/v1/organizations/1194841/devices/uplinks/addresses/byDevice
2025-09-21 13:07:58       meraki:     INFO > organizations, getOrganizationDevicesUplinksAddressesByDevice; page 1 - 200 OK


[{'mac': '08:f1:b3:e3:ae:65',
  'name': 'HOM_BIS-MV93',
  'network': {'id': 'L_575897802350019550'},
  'productType': 'camera',
  'serial': 'Q2LU-EDMJ-3X4W',
  'tags': [],
  'uplinks': [{'addresses': [{'address': '10.20.0.4',
                              'assignmentMode': 'dynamic',
                              'gateway': '10.20.0.1',
                              'nameservers': {'addresses': ['10.20.0.1']},
                              'protocol': 'ipv4',
                              'public': {'address': '212.105.155.226'},
                              'vlan': {'id': '0'}}],
               'interface': 'man1'}]}]


### Viene effettuata una correlazione tra dispositivi raggiungibili nella rete e dispositivi presenti. Dopodiché, vengono stampati i dispositivi rilevati ☑️

In [7]:
i = 0
UPLINK_DEVICE = {}
for device in response_uplink[0:3]:
    UPLINK_DEVICE[i] = {
        "NUMBER": i,
        "SERIAL": device['serial'],
        "DEVICE_PRODUCT": device['productType'],
        "DEVICE_UPLINK" : device['uplinks'] # Uplinks è un dizionario con due campi, indirizzi e interfacce. Indirizzi a sua volta è un dizionario con indirizzo pubblico, modalità di assegnamento, nameserver, interfaccia fisica, protocollo, indirizzo pubblico e vlan di appartenenza
    }
    i = i+1

for idx, device in DEVICE.items():
    serial = device["SERIAL"]
    uplink_info = next((uplink for uplink in UPLINK_DEVICE.values() if uplink["SERIAL"] == serial), None)
    if uplink_info:
        DEVICE[idx]["HAS_A_UPLINK"] = True
        DEVICE[idx]["DEVICE_UPLINK"] = uplink_info["DEVICE_UPLINK"]
    else:
        DEVICE[idx]["HAS_A_UPLINK"] = False
        DEVICE[idx]["DEVICE_UPLINK"] = None

pprint(DEVICE)

{0: {'DEVICE_MODEL': 'MV93',
     'DEVICE_PRODUCT': 'camera',
     'DEVICE_UPLINK': [{'addresses': [{'address': '10.20.0.4',
                                       'assignmentMode': 'dynamic',
                                       'gateway': '10.20.0.1',
                                       'nameservers': {'addresses': ['10.20.0.1']},
                                       'protocol': 'ipv4',
                                       'public': {'address': '212.105.155.226'},
                                       'vlan': {'id': '0'}}],
                        'interface': 'man1'}],
     'HAS_A_UPLINK': True,
     'MAC_ADDRESS': '08:f1:b3:e3:ae:65',
     'NUMBER': 0,
     'SERIAL': 'Q2LU-EDMJ-3X4W'},
 1: {'DEVICE_MODEL': 'MV93',
     'DEVICE_PRODUCT': 'camera',
     'DEVICE_UPLINK': None,
     'HAS_A_UPLINK': False,
     'MAC_ADDRESS': '08:f1:b3:e3:b0:15',
     'NUMBER': 1,
     'SERIAL': 'Q2LU-UZAF-F8ZH'},
 2: {'DEVICE_MODEL': 'MV93',
     'DEVICE_PRODUCT': 'camera',
     'DEVICE_UPLIN

### Vengono individuate le camere ☑️

In [8]:
# Trova il seriale di un dispositivo che è una camera
SERIAL_CAMERA = 'Q2LU-EDMJ-3X4W'


In [9]:
name = 'MQTT_Broker'
host = 'mqtt.merakiabitarein.app'
port = 80


### Impostazione MQTT broker nella rete(da fare una sola volta) ☑️

In [26]:

try:
    response = dashboard.networks.createNetworkMqttBroker(
        NETWORK_ID,
        name=name,
        host=host,
        port=port,
        security={'mode': 'none'},
        authentication={'username': 'abitarein', 'password': 'abitarein'}
    )
    print(response)
except meraki.APIError as e:
    print("Meraki API Error:", e)
    print("Response:", e.response.text)
except Exception as ex:
    print("General Exception:", ex)

2025-09-21 13:06:43       meraki:     INFO > POST https://api.meraki.com/api/v1/networks/L_575897802350018094/mqttBrokers
2025-09-21 13:06:43       meraki:     INFO > networks, createNetworkMqttBroker - 201 Created


{'id': '575897802350003148', 'name': 'MQTT_Broker', 'host': 'mqtt.merakiabitarein.app', 'port': 80, 'security': {'mode': 'none', 'tls': {'hasCaCertificate': False, 'verifyHostnames': True}}, 'authentication': {'username': 'abitarein'}}


### Imposta MVSense con MQTT Broker (Da fare una sola volta)

In [12]:
response_set_mvsense = dashboard.camera.updateDeviceCameraSense(
    SERIAL_CAMERA, 
    senseEnabled=True, 
    mqttBrokerId=cr.MQTT_BROKER_ID_THU, 
    audioDetection={'enabled': False}
)

print(response_set_mvsense)



2025-09-21 13:09:05       meraki:     INFO > PUT https://api.meraki.com/api/v1/devices/Q2LU-EDMJ-3X4W/camera/sense
2025-09-21 13:09:06       meraki:     INFO > camera, updateDeviceCameraSense - 200 OK


{'senseEnabled': True, 'mqttBrokerId': '575897802350003147', 'mqttTopics': ['/merakimv/Q2LU-EDMJ-3X4W/575897802350002810', '/merakimv/Q2LU-EDMJ-3X4W/raw_detections', '/merakimv/Q2LU-EDMJ-3X4W/light'], 'audioDetection': {'enabled': False}, 'detectionModelId': '1'}


#### Codice broker mqtt

In [14]:
import time

import paho.mqtt.client as mqtt

def start_meraki_camera_mqtt_client():
    topic = f"/testopic"
    broker = 'mqtt.merakiabitarein.app'
    port_num = 80
    username = "abitarein"
    password = "abitarein"

    def on_connect(client, userdata, flags, reason_code, properties):
        print(f"Connected with result code {reason_code}")
        # Subscribing in on_connect() means that if we lose the connection and
        # reconnect then subscriptions will be renewed.
        client.subscribe("$SYS/#")

    # The callback for when a PUBLISH message is received from the server.
    def on_message(client, userdata, msg):
        print(msg.topic+" "+str(msg.payload))

    client = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION2, client_id="pclient")
    client.username_pw_set(username, password)
    client.on_message = on_message
    client.on_connect = on_connect

    client.connect(broker, port_num,60)
    client.subscribe(topic)

    print(f"Subscribed to topic: {topic}")
    client.loop_start()
    print("MQTT client started.")

start_meraki_camera_mqtt_client()


try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Stopped MQTT client.")


Subscribed to topic: /testopic
MQTT client started.
Stopped MQTT client.


#### publisher mqtt


In [35]:
from random import randrange, uniform
import time

broker = "mqtt.merakiabitarein.app"
port = 80
client = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION2, client_id="prova")
client.username_pw_set("abitarein", "abitarein")
client.connect(broker, port)

while True:
    randNumber = uniform(20.0, 21.0)
    client.publish(topic = "/testopic",  payload = randNumber)
    print("published")
    time.sleep(1)


published
published
published
published
published
published
published
published
published
published
published
published
published
published
published
published
published
published
published
published
published
published
published


KeyboardInterrupt: 